## Red de Hopfield

In [6]:
import numpy as np

# Matriz de distancias entre ciudades
distancias = np.array([
    [0, 5, 5, 6, 4],
    [5, 0, 3, 7, 6],
    [5, 3, 0, 4, 8],
    [6, 7, 4, 0, 5],
    [4, 6, 8, 5, 0]
])

# Parámetros 
n_ciudades = 5
alpha = 100 
beta = 100   
gamma = 50  
max_iter = 1000

# (matriz n_ciudades x n_ciudades)
V = np.random.rand(n_ciudades, n_ciudades)

# Función(sigmoide)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

for iteracion in range(max_iter):

    nuevo_V = np.copy(V)
    
    for x in range(n_ciudades):      
        for i in range(n_ciudades):  
      
            suma = 0
            

            for y in range(n_ciudades):
                if y != x:
                    suma -= alpha * V[y, i]
            

            for j in range(n_ciudades):
                if j != i:
                    suma -= beta * V[x, j]
            
            # Minimización de distancia
            for y in range(n_ciudades):
                if y != x:
                    for j in range(n_ciudades):
                        if j != i:
                            # Distancia entre ciudad x y ciudad y
                            suma -= gamma * distancias[x, y] * V[y, j]
            
            nuevo_V[x, i] = sigmoid(suma + 2*n_ciudades)
    
    V = nuevo_V
    
    # Condición de parada opcional (cambio pequeño en V)
    if iteracion > 10 and np.max(np.abs(nuevo_V - V)) < 0.001:
        break

# ruta final
ruta = np.argmax(V, axis=1)

#  distancia total
distancia_total = sum(distancias[ruta[i], ruta[(i+1)%n_ciudades]] for i in range(n_ciudades))

print("Ruta encontrada:", ruta)
print("Distancia total:", distancia_total)

Ruta encontrada: [0 0 0 0 0]
Distancia total: 0


/var/folders/qh/5bmv1f155j34j21jkvcw3blh0000gn/T/ipykernel_6766/3027128586.py:24: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


## Solución dada por DeepSeek 

Este algoritmo dado por DeepSeek no encuentra solución a la matriz de distancias dada, como tambien se probó con el algoritmo previo

In [2]:
import numpy as np
import matplotlib.pyplot as plt

class HopfieldTSP:
    def __init__(self, distance_matrix, num_cities, alpha=100, beta=100, gamma=100, max_iter=1000):
        self.distance_matrix = distance_matrix
        self.num_cities = num_cities
        self.alpha = alpha  # Peso para la restricción de una ciudad por paso
        self.beta = beta    # Peso para la restricción de un paso por ciudad
        self.gamma = gamma  # Peso para la minimización de distancia
        self.max_iter = max_iter
        
    def initialize_network(self):
        # Inicialización aleatoria de los neuronios
        self.V = np.random.rand(self.num_cities, self.num_cities)
        
        # Calculamos los pesos de la red
        self.W = np.zeros((self.num_cities**2, self.num_cities**2))
        
        # Término constante para los bias
        self.bias = 2 * self.num_cities
        
        # Construimos la matriz de pesos
        for x in range(self.num_cities):
            for i in range(self.num_cities):
                for y in range(self.num_cities):
                    for j in range(self.num_cities):
                        if x == y and i != j:
                            self.W[x*self.num_cities+i, y*self.num_cities+j] = -self.alpha
                        if i == j and x != y:
                            self.W[x*self.num_cities+i, y*self.num_cities+j] = -self.beta
                        if x != y and i != j:
                            self.W[x*self.num_cities+i, y*self.num_cities+j] = -self.gamma * self.distance_matrix[x, y]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def energy(self, V):
        energy = 0
        V_flat = V.flatten()
        
        # Término cuadrático
        energy += 0.5 * np.dot(V_flat, np.dot(self.W, V_flat))
        
        # Término lineal (bias)
        energy += self.bias * np.sum(V_flat)
        
        return energy
    
    def solve(self):
        self.initialize_network()
        energy_history = []
        
        for _ in range(self.max_iter):
            # Actualización asíncrona de los neuronios
            for x in range(self.num_cities):
                for i in range(self.num_cities):
                    # Calculamos la entrada total al neuronio
                    total_input = np.sum(self.W[x*self.num_cities+i, :] * self.V.flatten()) + self.bias
                    
                    # Aplicamos la función de activación
                    self.V[x, i] = self.sigmoid(total_input)
            
            # Guardamos la energía en cada iteración
            energy_history.append(self.energy(self.V))
            
            # Verificamos convergencia
            if len(energy_history) > 10 and np.std(energy_history[-10:]) < 1e-5:
                break
        

        route = np.argmax(self.V, axis=1)
        

        if len(set(route)) == self.num_cities:
            return route, energy_history
        else:
            return None, energy_history
    
    def calculate_distance(self, route):
        total_distance = 0
        for i in range(self.num_cities - 1):
            total_distance += self.distance_matrix[route[i], route[i+1]]
        total_distance += self.distance_matrix[route[-1], route[0]] 
        return total_distance

# Matriz de distancias 
distance_matrix = np.array([
    [0, 5, 5, 6, 4],
    [5, 0, 3, 7, 6],
    [5, 3, 0, 4, 8],
    [6, 7, 4, 0, 5],
    [4, 6, 8, 5, 0]
])

# Resolvemos el problema
tsp_solver = HopfieldTSP(distance_matrix, num_cities=5, alpha=100, beta=100, gamma=50, max_iter=1000)
best_route, energy_history = tsp_solver.solve()

# Resultados
if best_route is not None:
    print(f"Mejor ruta encontrada: {best_route}")
    print(f"Distancia total: {tsp_solver.calculate_distance(best_route)}")
    
    # Visualización de la energía
    plt.plot(energy_history)
    plt.title("Convergencia de la Red de Hopfield")
    plt.xlabel("Iteración")
    plt.ylabel("Energía")
    plt.show()
    
    # Visualización de la ruta
    cities = ["A", "B", "C", "D", "E"]
    route_labels = [cities[i] for i in best_route] + [cities[best_route[0]]]
    print("Ruta óptima:", " → ".join(route_labels))
else:
    print("No se encontró una ruta válida. Intente con diferentes parámetros.")

No se encontró una ruta válida. Intente con diferentes parámetros.


/var/folders/qh/5bmv1f155j34j21jkvcw3blh0000gn/T/ipykernel_6766/170438641.py:36: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
